# Clustering des candidats jour par jour

### Imports

In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 999)
from sklearn.cluster import AffinityPropagation
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

import util

### Chargement des données

On importe les données et on ne garde que celles jusqu'au 1er tour.

In [2]:
df = util.import_data('../data/dataframe_full')
df = util.premier_tour(df)

On crée la liste $C$ des candidats et on désigne par $N$ le nombre de candidats.

In [3]:
C = ['Arthaud', 'Asselineau', 'Cheminade', 'Dupont-Aignan', 'Fillon',
     'Hamon','Lassalle','Le Pen','Macron','Melenchon','Poutou']
N = len(C) # le nombre de candidats

In [4]:
days = sorted(list(set(df['day'].values)))

Les paramètres de damping pour l'agorithme Affinity Propagation : [0.5, 0.6, 0.7, 0.8, 0.9]

In [6]:
damping_factors = [k/10 for k in range(5,10)]

### Calcul des similarités entre candidats
On définit la similarité entre deux candidats $c_1$ et $c_2$ par 
$ similarite(c_1,c_2) := \frac{\text{2 }\times \text{ nombre de tweets mentionnant $c_1$ et $c_2$}}{\text{nombre de tweets mentionnant $c_1$ + nombre de tweets mentionnant $c_2$}} $

Les matrices contenant les similarités sont rangées dans un dictionnaire $S$, où la matrice $S[j]$ est de dimension $N\times N$ telle que $S[k,l]$ donne la similarité entre le $k^{ème}$ et le $l^{ème}$ candidat de la liste $C$ pour le jour $j$. Pour calculer ces valeurs on calcule d'abord pour chaque candidat le nombre de tweets le mentionnant.

In [7]:
# dictionnaire contenant les matrices de similarités
S = { day:np.zeros(shape=(N,N)) for day in days}

# on itère sur les jours
for day in days:
    
    # on se retreint au jour en cours
    df_bis = df.loc[df['day']==day]

    # nombre de tweets pour chaque candidat pour ce jour-ci
    nb_tweets = { c:df_bis.loc[df_bis[c]==1].shape[0] for c in C }

    # calcul des similarités en itérant sur les tweets du jour
    for i,tweet in df_bis.iterrows():
        mentions = [c for c in C if tweet[c]==1]
        for c1 in mentions:
            for c2 in mentions:
                S[day][C.index(c1), C.index(c2)] += 2 / (nb_tweets[c1]+nb_tweets[c2])

### Clustering
Pour faire le clustering, on transforme les matrices de similarités en matrices de distances. Comme le max des matrices de similarités est 1 (valeurs de la diagonale) on définit la matrice des distances $D$ par $D[i,j] = 1 - S[i,j]$. On force 0 sur la diagonale parce que python met des valeurs $\approx 10^{-17}$ au lieu de 0.

In [8]:
D = dict()
for day in days:
    D[day] = np.full(shape=(N,N), fill_value=1) - S[day]
    np.fill_diagonal(D[day], 0)

D:\Anaconda\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((11, 11), 1) will return an array of dtype('int32')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


Maintenant on procède au clustering en utilisant l'algorithme Affinity Propagation avec la matrice de distances $D$ et on sauvegarde le résultat dans le dossier courant sous la forme d'un dictionnaire 

<center>$\text{{n°cluster : liste des membres du cluster}}$</center>

On a choisi de prendre un damping factor égal à 0.5 car il produit les résultats les plus intéressants.

In [9]:
clusters = dict()
for damp in damping_factors:
    for day in days:
        aff = AffinityPropagation(damping=damp, affinity='precomputed')
        aff_result = aff.fit_predict(D[day])
        clusters[(damp,day)] = { i : [c for c in C if aff_result[C.index(c)]==i] for i in set(aff_result) }

### Affichage des résultats

Pour valeur de damping, pour chaque jour, on affiche les clusters obtenus jour par jour.

In [10]:
for damp in damping_factors:
    print('DAMPING = ', damp)
    for day in days:
        print(day)
        for i in clusters[(damp,day)].keys():
            print(i,clusters[(damp,day)][i])
        print()
    print()

DAMPING =  0.5
2017-04-09
0 ['Cheminade', 'Hamon', 'Le Pen', 'Macron', 'Melenchon']
1 ['Arthaud', 'Asselineau', 'Dupont-Aignan', 'Fillon', 'Lassalle', 'Poutou']

2017-04-10
0 ['Arthaud', 'Dupont-Aignan', 'Hamon', 'Le Pen', 'Macron', 'Melenchon', 'Poutou']
1 ['Asselineau', 'Cheminade', 'Fillon', 'Lassalle']

2017-04-11
0 ['Arthaud', 'Asselineau', 'Cheminade', 'Fillon', 'Poutou']
1 ['Dupont-Aignan', 'Hamon', 'Lassalle', 'Le Pen', 'Macron', 'Melenchon']

2017-04-12
0 ['Cheminade', 'Fillon', 'Hamon', 'Le Pen', 'Melenchon']
1 ['Arthaud', 'Asselineau', 'Dupont-Aignan', 'Lassalle', 'Macron', 'Poutou']

2017-04-13
0 ['Cheminade', 'Hamon', 'Le Pen', 'Macron', 'Melenchon']
1 ['Arthaud', 'Asselineau', 'Dupont-Aignan', 'Fillon', 'Lassalle', 'Poutou']

2017-04-14
0 ['Cheminade', 'Fillon', 'Hamon', 'Le Pen', 'Melenchon']
1 ['Arthaud', 'Asselineau', 'Dupont-Aignan', 'Lassalle', 'Macron', 'Poutou']

2017-04-15
0 ['Arthaud', 'Cheminade', 'Dupont-Aignan', 'Hamon', 'Le Pen', 'Macron', 'Melenchon']
1 ['As

### APriori sur les clusters
Maintenant que nous avons les clusters jour par jour, on utilise l'agorithme APriori pour détecter les groupes de candidats qui sont fréquemments dans le même cluster. On ne prend pas en compte les clusters contenant tous les candidats (s'il y en a). Cela va nous permettre de reconnaître précisément les ensembles de candidats qui sont le plus cités ensemble. En sortie, on multiplie les supports par 2 pour avoir, pour chaque groupe de candidat fréquent, le pourcentage de jours où ils ont été placés dans le même cluster. On ne garde que les groupes de candidats dont la fréquence est au moins de $2/3$.

In [11]:
for damp in damping_factors:
    
    # on crée une liste contenant tous les clusters
    clusters_list = [ clusters[(damp,day)][i] for day in days for i in range(len(clusters[(damp,day)])) if len(clusters[(damp,day)][i])<N ] 

    # on encode
    te = TransactionEncoder()
    data_encoded = te.fit_transform(clusters_list)
    df_encoded = pd.DataFrame(data_encoded, columns=te.columns_)

    # APriori
    ap = apriori(df_encoded, min_support=0.1, use_colnames=True)
    ap = ap.sort_values(by='support', ascending=False)

    # on ne garde que les fréquents de taille >1 pour ne pas avoir les candidats seuls
    motifs = ap.values
    motifs = [ tuple(item) for item in motifs if len(item[1])>1 ]

    # si un motif a un sur-ensemble de support >= on le supprime
    to_remove = set() # va contenir les motifs à supprimer
    for m in motifs:
        for n in motifs:
            if m!=n and m[1].issubset(n[1]) and m[0]==n[0]:
                to_remove.add(m)
    motifs = set(motifs).difference(to_remove) # on enlève les motifs à supprimer

    # affichage des résultats
    ap = pd.DataFrame(list(motifs), columns=['fréquence','groupe'])
    ap['fréquence'] = ap['fréquence'].apply(lambda x: x*2)
    ap = ap.loc[ap['fréquence']>0.66]
    ap = ap.sort_values(by='fréquence', ascending=False)
    print('------------------ DAMPING = {} ------------------'.format(damp))
    print(ap.to_string(index=False, header=False))
    print()

------------------ DAMPING = 0.5 ------------------
1.000000                       (Hamon, Melenchon)
0.916667                     (Poutou, Asselineau)
0.916667                   (Asselineau, Lassalle)
0.916667                        (Poutou, Arthaud)
0.833333           (Poutou, Asselineau, Lassalle)
0.833333            (Poutou, Asselineau, Arthaud)
0.750000                 (Dupont-Aignan, Arthaud)
0.750000               (Hamon, Macron, Melenchon)
0.750000  (Poutou, Asselineau, Arthaud, Lassalle)
0.666667               (Hamon, Le Pen, Melenchon)
0.666667            (Hamon, Cheminade, Melenchon)
0.666667                      (Le Pen, Cheminade)
0.666667                  (Dupont-Aignan, Macron)
0.666667                (Dupont-Aignan, Lassalle)
0.666667         (Dupont-Aignan, Poutou, Arthaud)

------------------ DAMPING = 0.6 ------------------
1.000000                       (Hamon, Melenchon)
0.916667                     (Poutou, Asselineau)
0.916667                   (Asselineau, Lassa